# Proyecto final - Minería de Datos
### Alejandro Carregha, Eduardo Don Pablo y José Salvador Abascal

## Descripción del Dataset

El conjunto de datos **Sustainable Sports Event Management (SSEM)** es una base de información diseñada para apoyar investigaciones y análisis relacionados con la organización sostenible de eventos deportivos. Incluye registros detallados y características clave para evaluar la sostenibilidad, el impacto social, el uso eficiente de recursos y distintos tipos de eventos. Ofrece una visión completa de diversos aspectos de los eventos deportivos, considerando factores ambientales, sociales y económicos.

### Variables del dataset:
##### Variables de sustentabilidad:
- **Energy Consumption**: Categórico (Bajo, Moderado, Alto). Indica el nivel de energía que requiere cada evento.
- **Carbon Emissions**: Categórico (Bajas, Moderadas, Altas). Representa la huella de carbono generada.
- **Waste Generation**: Categórico (Baja, Moderada, Alta). Mide la cantidad de residuos producidos, útil para evaluar la gestión de desechos.

##### Variables de impacto social:
- **Community Engagement**: Categórico (Baja, Media, Alta). Evalúa qué tanto se involucra la comunidad en cada evento.
- **Volunteer Participation**: Categórico (Baja, Moderada, Alta). Mide cuántas personas colaboran voluntariamente.
- **Health Impact**: Categórico (Bajo, Moderado, Alto). Evalúa el efecto positivo del evento en la salud de participantes y asistentes.
- **Water Usage**: Categórico (Bajo, Moderado, Alto). Refleja cuánta agua se consume en cada evento.
- **Material Recycling Rate**: Categórico (Baja, Moderada, Alta). Mide qué tanto se reciclan los materiales utilizados.
- **Operational Cost Efficiency**: Categórico (Baja, Moderada, Alta). Evalúa qué tan eficiente es el uso del presupuesto del evento.

##### Variables de tipo de evento:
- **Event Scale**: Categórico (Local, Regional, Nacional). Indica el alcance geográfico del evento.
- **Event Focus**: Categórico (Salud, Juventud, Desarrollo Comunitario, Recreación). Describe el objetivo principal del evento.

##### Variables objetivo:
- **Sustainability Score**: Categórico (Bajo, Moderado, Alto). Resume el nivel general de sostenibilidad del evento.
- **Nivel de Impacto Social**: Categórico (Bajo, Moderado, Alto). Indica los beneficios sociales y comunitarios del evento.
- **Resource Efficiency**: Categórico (Baja, Moderada, Alta). Evalúa el uso eficiente de recursos.
- **Event Type Classification**: Categórico (Salud, Juventud, Desarrollo Comunitario, Recreación). Permite categorizar el evento según su enfoque.

El objetivo es predecir estas distintas variables objetivo categóricas basándose en las distintas ramas de variables de predicción. En este proyecto se usarán métodos como bosques y árboles de decisión, regresiones lineales (normales y penalizadas) y métodos de *boosting*.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [2]:
# Descargar y leer el dataset
data = "/Users/abascalleonjosesalvador/Downloads/sports_management_dataset.csv"
advertising = pd.read_csv(data, index_col=0)

print("Dataset cargado exitosamente!")
print(f"Dimensiones del dataset: {advertising.shape}")
print(f"Columnas: {advertising.columns.tolist()}")

Dataset cargado exitosamente!
Dimensiones del dataset: (102000, 14)
Columnas: ['Carbon Emissions', 'Waste Generation', 'Community Engagement', 'Volunteer Participation', 'Health Impact', 'Water Usage', 'Material Recycling Rate', 'Operational Cost Efficiency', 'Event Scale', 'Event Focus', 'Sustainability Score', 'Social Impact Level', 'Resource Efficiency', 'Event Type Classification']
